In [1]:
import pandas as pd
import os, time
import numpy as np

In [2]:
def timeit(method):
    '''A time decorator to time other functions.'''
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' %(method.__name__, (te - ts) * 1000))
        return result
    return timed

@timeit
def tableClean(file):
    '''Specifically build to clean leavetime table. Drop useless rows and columns to minimize the size of the file.'''
    dropCols = ['datasource','passengers','passengersin','passengersout','distance','note','lastupdate','justificationid',
                'plannedtime_dep', 'vehicleid']
    df = pd.DataFrame()
    for chunk in pd.read_csv(file, sep=';', chunksize=10000):
        chunk = chunk.drop(dropCols,1)
        chunk = chunk.drop(chunk[chunk.suppressed >= 0].index)
        chunk = chunk.drop(['suppressed'],1)
        chunk['dayofservice'] = pd.to_datetime(chunk['dayofservice'], format='%d-%b-%y %H:%M:%S')
        chunk = chunk.rename({'plannedtime_arr' : 'plannedtime'}, axis=1)
        #leavetimesOf39aSample = pd.concat([leavetimesOf39aSample,chunk.loc[chunk['tripid'] == trip]])
        df = pd.concat([df,chunk])
    df.to_csv(file+".csv", index=False)

In [ ]:
#tableClean('rt_leavetimes_2016_I_DB.txt')
#This line above kill the kernel because it dried all the resources. So, I bash again.
#awk -F";" 'BEGIN{OFS=",";} { print $2,$3,$4,$5,$6,$8,$9,$15 >> "2016.csv"}' ../rt_leavetimes_2016_I_DB.txt
# This line above drop all the useless columns. Next step: date time, drop suppressed and rename.
# This line shorten the time 00:00:00:
# awk -F, -vOFS=, 'NR>1{$1=substr($1,1,9)} { print $0 >> "2016t.csv" }' 2016.csv
# This line will drop suppressed flaged
#  grep ',$' 2016t.csv >> 2016s.csv (Explain: any line doesn't end with ,$ means there is something in that column)
# use this one to keep header: awk -F"," -vOFS="," 'NR==1 || (NR>1 && $8=="")' t >>t.csv
# This line above has a flaw that it remove my header too.
# remove last column
# awk -F"," 'BEGIN{OFS=",";} NF{NF-=1};1' <2016s.csv >>2016r.csv
# insert header.
#  sed -i '1 i\dayofservice,tripid,progrnumber,stopid,plannedtime,actual_arr,actual_dep\n' 2016r.csv
# It takes quite a while which makes me worry either it's destroying, or it need to move all the location backward which is time
# consuming!
# Ok, don't do the \n thing, it add an empty line ffs.
# this line remove empty rows: sed -i '/^$/d' 2016r.csv &

@timeit
def createLeavetime(routeid,bigFile):
    '''Extract leavetime table base on different routeids.'''
    path = 'Test/tripids/'
    #from numpy import genfromtxt
    #my_data = genfromtxt('my_file.csv', delimiter=',')
    tripids = np.genfromtxt(path+routeid)
    #print(tripids)
    temp = pd.DataFrame()
    for chunk in pd.read_csv(bigFile, chunksize=10000):
        for trip in tripids:
            temp = pd.concat([temp,chunk.loc[chunk['tripid'] == trip]])
    temp.to_csv('separated/'+routeid+'.csv',index=False)
                

for file in os.listdir('Test/tripids/'):
    createLeavetime(file,'2016leavetimes.csv')

Still too slow.... Bash time!